In [1]:
import torch
import torchvision

# Load a pre-trained version of MobileNetV2
torch_model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)
torch_model.eval()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /Users/sseibert/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [2]:
# Trace with random data
example_input = torch.rand(1, 3, 224, 224) # after test, will get 'size mismatch' error message with size 256x256
traced_model = torch.jit.trace(torch_model, example_input)

In [3]:
# Download class labels (from a separate file)
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()

class_labels = class_labels[1:] # remove the first class which is background
assert len(class_labels) == 1000

In [4]:
import coremltools as ct
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input_1", shape=example_input.shape)], #name "input_1" is used in 'quickstart'
    classifier_config = ct.ClassifierConfig(class_labels) # provide only if step 4 was performed
)

Translating MIL ==> MLModel Ops: 100%|██████████| 706/706 [00:00<00:00, 1976.16 ops/s]


In [5]:
# Save model
model.save("MobileNetV2.mlmodel")